# SQL Query Metrics

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
from ragas.llms import LangchainLLMWrapper

groq_llm = ChatGroq(model="llama3-8b-8192")
evaluator_llm = LangchainLLMWrapper(groq_llm)

d:\My Files\RAG_with_RAGAS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Execution based metrics

- the resulting SQL is compared after executing the SQL query on the database and then comparing the response with the expected results.

#### DataCompyScore

- DataCompyScore metric uses DataCompy, a python library that compares two pandas DataFrames. 
- It provides a simple interface to compare two DataFrames and provides a detailed report of the differences. 
- In this metric the response is executed on the database and the resulting data is compared with the expected data, ie reference.

In [8]:
from ragas.metrics import DataCompyScore
from ragas.dataset_schema import SingleTurnSample

data1 = """acct_id,dollar_amt,name,float_fld,date_fld
10000001234,123.45,George Maharis,14530.1555,2017-01-01
10000001235,0.45,Michael Bluth,1,2017-01-01
10000001236,1345,George Bluth,,2017-01-01
10000001237,123456,Bob Loblaw,345.12,2017-01-01
10000001238,1.05,Lucille Bluth,,2017-01-01
10000001238,1.05,Loose Seal Bluth,,2017-01-01
"""

data2 = """acct_id,dollar_amt,name,float_fld
10000001234,123.4,George Michael Bluth,14530.155
10000001235,0.45,Michael Bluth,
10000001236,1345,George Bluth,1
10000001237,123456,Robert Loblaw,345.12
10000001238,1.05,Loose Seal Bluth,111
"""
sample = SingleTurnSample(response=data1, reference=data2)
scorer = DataCompyScore(mode="rows", metric="recall")
await scorer.single_turn_ascore(sample)


np.float64(0.0)

## Non Execution based metrics

#### SQL Query Semantic equivalence

- LLMSQLEquivalence is a metric that can be used to evaluate the equivalence of response query with reference query. 
- The metric also needs database schema to be used when comparing queries, this is inputted in reference_contexts.

In [9]:
from ragas.metrics import LLMSQLEquivalence
from ragas.dataset_schema import SingleTurnSample

sample = SingleTurnSample(
    response="""
        SELECT p.product_name, SUM(oi.quantity) AS total_quantity
        FROM order_items oi
        JOIN products p ON oi.product_id = p.product_id
        GROUP BY p.product_name;
    """,
    reference="""
        SELECT p.product_name, COUNT(oi.quantity) AS total_quantity
        FROM order_items oi
        JOIN products p ON oi.product_id = p.product_id
        GROUP BY p.product_name;
    """,
    reference_contexts=[
        """
        Table order_items:
        - order_item_id: INT
        - order_id: INT
        - product_id: INT
        - quantity: INT
        """,
        """
        Table products:
        - product_id: INT
        - product_name: VARCHAR
        - price: DECIMAL
        """
    ]
)

scorer = LLMSQLEquivalence(llm= evaluator_llm)
await scorer.single_turn_ascore(sample)

0